In [19]:
import requests
import pandas as pd
import sqlite3
import numpy as np

## Raw Data Pull and DB Creation

In [20]:
#API Setup
BASE_URL = 'https://api.collegefootballdata.com/'
API_KEY = 'Y2P4Ex6vaj/fPBURQsf2jz+0R2pXikYv8PtvqoqiMG7ukTvpVscCVjUA10VDv+My'

def get_data(endpoint,params={}):
    headers = {"Authorization": f"Bearer {API_KEY}"}
    response = requests.get(f"{BASE_URL}/{endpoint}", headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"API Error: {response.status_code} - {response.text}")
    return response.json()

In [21]:
# Initial Data Pull (Games)
games_data = []
for year in range(2013, 2025):
    print(f"Pulling games for {year}...")
    data = get_data("games", {"year": year, "division": "fbs"})
    games_data.extend(data)
games_df = pd.DataFrame(games_data)
games_df = games_df.drop(['home_line_scores','away_line_scores'], axis=1)
games_df.head()  # Quick check

Pulling games for 2013...
Pulling games for 2014...
Pulling games for 2015...
Pulling games for 2016...
Pulling games for 2017...
Pulling games for 2018...
Pulling games for 2019...
Pulling games for 2020...
Pulling games for 2021...
Pulling games for 2022...
Pulling games for 2023...
Pulling games for 2024...


,id,season,week,season_type,start_date,start_time_tbd,completed,neutral_site,conference_game,attendance,...,away_team,away_conference,away_division,away_points,away_post_win_prob,away_pregame_elo,away_postgame_elo,excitement_index,highlights,notes
0,332412579,2013,1,regular,2013-08-29T22:00:00.000Z,None,True,False,False,81572.0,...,North Carolina,ACC,fbs,10.0,0.3444138448668871,1638.0,1612.0,None,None,None
1,332412309,2013,1,regular,2013-08-29T22:00:00.000Z,False,True,False,False,20790.0,...,Liberty,Big South,fcs,10.0,0.6038425390589921,1467.0,1461.0,None,None,None
2,332410154,2013,1,regular,2013-08-29T22:30:00.000Z,False,True,False,False,26202.0,...,Presbyterian,Big South,fcs,7.0,0.00021090286491187715,NaN,NaN,None,None,None
3,332412050,2013,1,regular,2013-08-29T23:00:00.000Z,False,True,False,False,16327.0,...,Illinois State,MVFC,fcs,28.0,0.086375149540862,NaN,NaN,None,None,None
4,332410189,2013,1,regular,2013-08-29T23:00:00.000Z,None,True,False,False,18142.0,...,Tulsa,Conference USA,fbs,7.0,0.35146523607074265,1635.0,1542.0,None,None,None


In [22]:
#SQLite Setup
conn = sqlite3.connect("cfb_data.db")
games_df.to_sql("games", conn, if_exists="replace", index=False)
conn.close()

In [23]:
#Pull Lines Data
lines_data = []
for year in range(2013, 2025):
    print(f"Pulling lines for {year}...")
    data = get_data("lines", {"year": year, "division": "fbs"})
    lines_data.extend(data)

# Flatten the nested structure
flat_lines = []
for game in lines_data:
    game_id = game['id']
    home_team = game['homeTeam']
    away_team = game['awayTeam']
    if game['lines']:  # Check if lines exist
        for line in game['lines']:
            flat_lines.append({
                'game_id': game_id,
                'home_team': home_team,
                'away_team': away_team,
                'sportsbook': line['provider'],
                'spread_open': line.get('spreadOpen', None),
                'spread': line.get('spread', None)  # Closing spread
            })

lines_df = pd.DataFrame(flat_lines)
print(lines_df.shape)
lines_df.head()  # Check it out

Pulling lines for 2013...
Pulling lines for 2014...
Pulling lines for 2015...
Pulling lines for 2016...
Pulling lines for 2017...
Pulling lines for 2018...
Pulling lines for 2019...
Pulling lines for 2020...
Pulling lines for 2021...
Pulling lines for 2022...
Pulling lines for 2023...
Pulling lines for 2024...
(33390, 6)


,game_id,home_team,away_team,sportsbook,spread_open,spread
0,332412579,South Carolina,North Carolina,teamrankings,None,-11
1,332412579,South Carolina,North Carolina,numberfire,None,-11
2,332412579,South Carolina,North Carolina,consensus,None,-11.5
3,332410062,Hawai'i,USC,teamrankings,None,23.5
4,332410062,Hawai'i,USC,numberfire,None,23.5


In [25]:
#Save to SQLite
conn = sqlite3.connect("cfb_data.db")
lines_df.to_sql("lines", conn, if_exists="replace", index=False)
conn.close()

In [26]:
# Average Spreads and Update Games Table
conn = sqlite3.connect("cfb_data.db")

# Query to average closing spreads
query = """
SELECT 
    g.*, 
    AVG(l.spread) AS avg_closing_spread
FROM 
    games g
LEFT JOIN 
    lines l ON g.id = l.game_id
GROUP BY 
    g.id, g.season, g.week, g.start_date, g.home_team, g.home_points, 
    g.away_team, g.away_points
"""

# Load into DataFrame
merged_df = pd.read_sql_query(query, conn)

# Overwrite games table with new data (no spread_open)
merged_df.to_sql("games", conn, if_exists="replace", index=False)

# Verify
print(merged_df.shape)
merged_df.head()

conn.close()

(9817, 32)


In [31]:
# Cell 9: Pull Advanced Stats
advanced_stats_data = []
for year in range(2013, 2025):
    print(f"Pulling advanced stats for {year}...")
    data = get_data("stats/game/advanced", {"year": year, "excludeGarbageTime":"true", "division": "fbs"})
    advanced_stats_data.extend(data)

Pulling advanced stats for 2013...
Pulling advanced stats for 2014...
Pulling advanced stats for 2015...
Pulling advanced stats for 2016...
Pulling advanced stats for 2017...
Pulling advanced stats for 2018...
Pulling advanced stats for 2019...
Pulling advanced stats for 2020...
Pulling advanced stats for 2021...
Pulling advanced stats for 2022...
Pulling advanced stats for 2023...
Pulling advanced stats for 2024...
(24136, 6)


,gameId,week,team,opponent,offense,defense
0,332410006,1,South Alabama,Southern Utah,"{'plays': 61, 'drives': 13, 'ppa': 0.034285508...","{'plays': 62, 'drives': 12, 'ppa': 0.181068714..."
1,332410006,1,Southern Utah,South Alabama,"{'plays': 62, 'drives': 12, 'ppa': 0.181068714...","{'plays': 61, 'drives': 13, 'ppa': 0.034285508..."
2,332410023,1,Sacramento State,San José State,"{'plays': 60, 'drives': 10, 'ppa': -0.03027895...","{'plays': 45, 'drives': 9, 'ppa': 0.0051840896..."
3,332410023,1,San José State,Sacramento State,"{'plays': 45, 'drives': 9, 'ppa': 0.0051840896...","{'plays': 60, 'drives': 10, 'ppa': -0.03027895..."
4,332410041,1,Towson,UConn,"{'plays': 64, 'drives': 13, 'ppa': 0.033970617...","{'plays': 48, 'drives': 13, 'ppa': 0.107786902..."


In [37]:
# Flatten the nested JSON
flat_stats = []
for game in advanced_stats_data:
    row = {
        'gameId': game['gameId'],
        'week': game['week'],
        'team': game['team'],
        'opponent': game['opponent']
    }
    # Flatten top-level offense stats
    for key, value in game['offense'].items():
        if isinstance(value, dict):  # Subkeys like standardDowns
            for subkey, subvalue in value.items():
                row[f'offense_{key}_{subkey}'] = subvalue
        else:
            row[f'offense_{key}'] = value
    # Flatten top-level defense stats
    for key, value in game['defense'].items():
        if isinstance(value, dict):  # Subkeys like passingPlays
            for subkey, subvalue in value.items():
                row[f'defense_{key}_{subkey}'] = subvalue
        else:
            row[f'defense_{key}'] = value
    flat_stats.append(row)


# Create DataFrame
advanced_stats_df = pd.DataFrame(flat_stats)
print(advanced_stats_df.shape)
advanced_stats_df.head()  # Check the data

(24136, 60)


,gameId,week,team,opponent,offense_plays,offense_drives,offense_ppa,offense_totalPPA,offense_successRate,offense_explosiveness,...,defense_passingDowns_successRate,defense_passingDowns_explosiveness,defense_rushingPlays_ppa,defense_rushingPlays_totalPPA,defense_rushingPlays_successRate,defense_rushingPlays_explosiveness,defense_passingPlays_ppa,defense_passingPlays_totalPPA,defense_passingPlays_successRate,defense_passingPlays_explosiveness
0,332410006,1,South Alabama,Southern Utah,61,13,0.034286,2.091416,0.377049,0.933890,...,0.307692,1.753290,0.172830,6.221864,0.277778,1.285971,0.327380,7.529743,0.347826,1.833458
1,332410006,1,Southern Utah,South Alabama,62,12,0.181069,11.226260,0.338710,1.190573,...,0.210526,2.559272,-0.056888,-1.934180,0.382353,0.588519,0.149096,4.025596,0.370370,1.382872
2,332410023,1,Sacramento State,San José State,60,10,-0.030279,-1.816737,0.416667,0.861106,...,0.214286,1.575594,0.098626,1.873889,0.315789,0.791780,-0.077537,-1.938417,0.320000,1.288211
3,332410023,1,San José State,Sacramento State,45,9,0.005184,0.233284,0.333333,1.023612,...,0.526316,1.237267,-0.170181,-4.765059,0.321429,0.652246,0.092135,2.948322,0.500000,0.978589
4,332410041,1,Towson,UConn,64,13,0.033971,2.174119,0.328125,1.366551,...,0.375000,1.892898,0.014553,0.320163,0.363636,0.843752,0.232041,5.801016,0.400000,1.617091


In [39]:
advanced_stats_df.columns.tolist()

['gameId',
 'week',
 'team',
 'opponent',
 'offense_plays',
 'offense_drives',
 'offense_ppa',
 'offense_totalPPA',
 'offense_successRate',
 'offense_explosiveness',
 'offense_powerSuccess',
 'offense_stuffRate',
 'offense_lineYards',
 'offense_lineYardsTotal',
 'offense_secondLevelYards',
 'offense_secondLevelYardsTotal',
 'offense_openFieldYards',
 'offense_openFieldYardsTotal',
 'offense_standardDowns_ppa',
 'offense_standardDowns_successRate',
 'offense_standardDowns_explosiveness',
 'offense_passingDowns_ppa',
 'offense_passingDowns_successRate',
 'offense_passingDowns_explosiveness',
 'offense_rushingPlays_ppa',
 'offense_rushingPlays_totalPPA',
 'offense_rushingPlays_successRate',
 'offense_rushingPlays_explosiveness',
 'offense_passingPlays_ppa',
 'offense_passingPlays_totalPPA',
 'offense_passingPlays_successRate',
 'offense_passingPlays_explosiveness',
 'defense_plays',
 'defense_drives',
 'defense_ppa',
 'defense_totalPPA',
 'defense_successRate',
 'defense_explosiveness',
 

In [41]:
# Cell 10: Save to SQLite
conn = sqlite3.connect("cfb_data.db")
advanced_stats_df.to_sql("advanced_stats", conn, if_exists="replace", index=False)
conn.close()

In [ ]:
# Merge Games and Advanced Stats (All Columns)
conn = sqlite3.connect("cfb_data.db")

query = """
SELECT 
    g.*,
    h.offense_plays AS home_offense_plays,
    h.offense_drives AS home_offense_drives,
    h.offense_ppa AS home_offense_ppa,
    h.offense_totalPPA AS home_offense_totalPPA,
    h.offense_successRate AS home_offense_successRate,
    h.offense_explosiveness AS home_offense_explosiveness,
    h.offense_powerSuccess AS home_offense_powerSuccess,
    h.offense_stuffRate AS home_offense_stuffRate,
    h.offense_lineYards AS home_offense_lineYards,
    h.offense_lineYardsTotal AS home_offense_lineYardsTotal,
    h.offense_secondLevelYards AS home_offense_secondLevelYards,
    h.offense_secondLevelYardsTotal AS home_offense_secondLevelYardsTotal,
    h.offense_openFieldYards AS home_offense_openFieldYards,
    h.offense_openFieldYardsTotal AS home_offense_openFieldYardsTotal,
    h.offense_standardDowns_ppa AS home_offense_standardDowns_ppa,
    h.offense_standardDowns_successRate AS home_offense_standardDowns_successRate,
    h.offense_standardDowns_explosiveness AS home_offense_standardDowns_explosiveness,
    h.offense_passingDowns_ppa AS home_offense_passingDowns_ppa,
    h.offense_passingDowns_successRate AS home_offense_passingDowns_successRate,
    h.offense_passingDowns_explosiveness AS home_offense_passingDowns_explosiveness,
    h.offense_rushingPlays_ppa AS home_offense_rushingPlays_ppa,
    h.offense_rushingPlays_totalPPA AS home_offense_rushingPlays_totalPPA,
    h.offense_rushingPlays_successRate AS home_offense_rushingPlays_successRate,
    h.offense_rushingPlays_explosiveness AS home_offense_rushingPlays_explosiveness,
    h.offense_passingPlays_ppa AS home_offense_passingPlays_ppa,
    h.offense_passingPlays_totalPPA AS home_offense_passingPlays_totalPPA,
    h.offense_passingPlays_successRate AS home_offense_passingPlays_successRate,
    h.offense_passingPlays_explosiveness AS home_offense_passingPlays_explosiveness,
    h.defense_plays AS home_defense_plays,
    h.defense_drives AS home_defense_drives,
    h.defense_ppa AS home_defense_ppa,
    h.defense_totalPPA AS home_defense_totalPPA,
    h.defense_successRate AS home_defense_successRate,
    h.defense_explosiveness AS home_defense_explosiveness,
    h.defense_powerSuccess AS home_defense_powerSuccess,
    h.defense_stuffRate AS home_defense_stuffRate,
    h.defense_lineYards AS home_defense_lineYards,
    h.defense_lineYardsTotal AS home_defense_lineYardsTotal,
    h.defense_secondLevelYards AS home_defense_secondLevelYards,
    h.defense_secondLevelYardsTotal AS home_defense_secondLevelYardsTotal,
    h.defense_openFieldYards AS home_defense_openFieldYards,
    h.defense_openFieldYardsTotal AS home_defense_openFieldYardsTotal,
    h.defense_standardDowns_ppa AS home_defense_standardDowns_ppa,
    h.defense_standardDowns_successRate AS home_defense_standardDowns_successRate,
    h.defense_standardDowns_explosiveness AS home_defense_standardDowns_explosiveness,
    h.defense_passingDowns_ppa AS home_defense_passingDowns_ppa,
    h.defense_passingDowns_successRate AS home_defense_passingDowns_successRate,
    h.defense_passingDowns_explosiveness AS home_defense_passingDowns_explosiveness,
    h.defense_rushingPlays_ppa AS home_defense_rushingPlays_ppa,
    h.defense_rushingPlays_totalPPA AS home_defense_rushingPlays_totalPPA,
    h.defense_rushingPlays_successRate AS home_defense_rushingPlays_successRate,
    h.defense_rushingPlays_explosiveness AS home_defense_rushingPlays_explosiveness,
    h.defense_passingPlays_ppa AS home_defense_passingPlays_ppa,
    h.defense_passingPlays_totalPPA AS home_defense_passingPlays_totalPPA,
    h.defense_passingPlays_successRate AS home_defense_passingPlays_successRate,
    h.defense_passingPlays_explosiveness AS home_defense_passingPlays_explosiveness,
    a.offense_plays AS away_offense_plays,
    a.offense_drives AS away_offense_drives,
    a.offense_ppa AS away_offense_ppa,
    a.offense_totalPPA AS away_offense_totalPPA,
    a.offense_successRate AS away_offense_successRate,
    a.offense_explosiveness AS away_offense_explosiveness,
    a.offense_powerSuccess AS away_offense_powerSuccess,
    a.offense_stuffRate AS away_offense_stuffRate,
    a.offense_lineYards AS away_offense_lineYards,
    a.offense_lineYardsTotal AS away_offense_lineYardsTotal,
    a.offense_secondLevelYards AS away_offense_secondLevelYards,
    a.offense_secondLevelYardsTotal AS away_offense_secondLevelYardsTotal,
    a.offense_openFieldYards AS away_offense_openFieldYards,
    a.offense_openFieldYardsTotal AS away_offense_openFieldYardsTotal,
    a.offense_standardDowns_ppa AS away_offense_standardDowns_ppa,
    a.offense_standardDowns_successRate AS away_offense_standardDowns_successRate,
    a.offense_standardDowns_explosiveness AS away_offense_standardDowns_explosiveness,
    a.offense_passingDowns_ppa AS away_offense_passingDowns_ppa,
    a.offense_passingDowns_successRate AS away_offense_passingDowns_successRate,
    a.offense_passingDowns_explosiveness AS away_offense_passingDowns_explosiveness,
    a.offense_rushingPlays_ppa AS away_offense_rushingPlays_ppa,
    a.offense_rushingPlays_totalPPA AS away_offense_rushingPlays_totalPPA,
    a.offense_rushingPlays_successRate AS away_offense_rushingPlays_successRate,
    a.offense_rushingPlays_explosiveness AS away_offense_rushingPlays_explosiveness,
    a.offense_passingPlays_ppa AS away_offense_passingPlays_ppa,
    a.offense_passingPlays_totalPPA AS away_offense_passingPlays_totalPPA,
    a.offense_passingPlays_successRate AS away_offense_passingPlays_successRate,
    a.offense_passingPlays_explosiveness AS away_offense_passingPlays_explosiveness,
    a.defense_plays AS away_defense_plays,
    a.defense_drives AS away_defense_drives,
    a.defense_ppa AS away_defense_ppa,
    a.defense_totalPPA AS away_defense_totalPPA,
    a.defense_successRate AS away_defense_successRate,
    a.defense_explosiveness AS away_defense_explosiveness,
    a.defense_powerSuccess AS away_defense_powerSuccess,
    a.defense_stuffRate AS away_defense_stuffRate,
    a.defense_lineYards AS away_defense_lineYards,
    a.defense_lineYardsTotal AS away_defense_lineYardsTotal,
    a.defense_secondLevelYards AS away_defense_secondLevelYards,
    a.defense_secondLevelYardsTotal AS away_defense_secondLevelYardsTotal,
    a.defense_openFieldYards AS away_defense_openFieldYards,
    a.defense_openFieldYardsTotal AS away_defense_openFieldYardsTotal,
    a.defense_standardDowns_ppa AS away_defense_standardDowns_ppa,
    a.defense_standardDowns_successRate AS away_defense_standardDowns_successRate,
    a.defense_standardDowns_explosiveness AS away_defense_standardDowns_explosiveness,
    a.defense_passingDowns_ppa AS away_defense_passingDowns_ppa,
    a.defense_passingDowns_successRate AS away_defense_passingDowns_successRate,
    a.defense_passingDowns_explosiveness AS away_defense_passingDowns_explosiveness,
    a.defense_rushingPlays_ppa AS away_defense_rushingPlays_ppa,
    a.defense_rushingPlays_totalPPA AS away_defense_rushingPlays_totalPPA,
    a.defense_rushingPlays_successRate AS away_defense_rushingPlays_successRate,
    a.defense_rushingPlays_explosiveness AS away_defense_rushingPlays_explosiveness,
    a.defense_passingPlays_ppa AS away_defense_passingPlays_ppa,
    a.defense_passingPlays_totalPPA AS away_defense_passingPlays_totalPPA,
    a.defense_passingPlays_successRate AS away_defense_passingPlays_successRate,
    a.defense_passingPlays_explosiveness AS away_defense_passingPlays_explosiveness
FROM 
    games g
LEFT JOIN 
    advanced_stats h ON g.id = h.gameId AND g.home_team = h.team
LEFT JOIN 
    advanced_stats a ON g.id = a.gameId AND g.away_team = a.team
"""

# Load into DataFrame
games_with_stats_df = pd.read_sql_query(query, conn)

# Save as new table
games_with_stats_df.to_sql("games_with_stats", conn, if_exists="replace", index=False)

print(games_with_stats_df.shape)

conn.close()

(9817, 144)


In [45]:
games_with_stats_df.to_excel('dataset.xlsx')